In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2b1e1106-23a4-7428-55c2-c5428daa9252)


In [19]:
!pip install transformers
!pip install simpletransformers
!pip install scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -lrt /content/drive/"My Drive"/SDE_Project/AllSentences2.csv

-rw------- 1 root root 511113 Mar  2 20:42 '/content/drive/My Drive/SDE_Project/AllSentences2.csv'


In [22]:
import numpy as np
import pandas as pd
import re
import random,os
import torch
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split

seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

data_path = '/content/drive/My Drive/SDE_Project/AllSentences2.csv'
all_sentences = pd.read_csv(data_path, sep=",")
all_sentences.fillna(value='', inplace=True)

### The pre-processing steps, labelling and train-test split as setup to be similar to the authors implementation. This is to ensure reproducibility

def clean_sent(row):
  return re.sub(r'([a-z])([A-Z])', r'\1 \2', str(row['sentence']))

def check_param(sentences, i):
  return ("_" in sentences['sentence'].iloc[i])

def get_context(cur_sentences, i):
  for k in range(i, -1, -1):
    if check_param(cur_sentences, k):
      return k
  return 0

def get_vals(sentence):
  sentence_cands = []
  sentence_clean = re.sub(',|;|:|"|“|\u201F|\u201D|\u201C|\u0022\)|\(', 
                     ' ', sentence)
  for sent in sentence_clean.split():
    if re.match(r'\d+.*|on$|off$', sent):
      sentence_cands.append(sent)
  return sentence_cands

def get_params(sentence):
  sentence_clean = re.sub(r',|;|:|"|“|”|\)|\(', ' ', sentence)
  sentence_cands = []
  for sent in sentence_clean.split():
    if "_" in sent:
      sentence_cands.append(sent)
  return sentence_cands

def label_formula_ops(row):
  str_form = str(row['Formula'])
  if "<" in str_form and ">" in str_form:
    return 0
  if "<" in str_form:
    return 1
  elif ">" in str_form:
    return 2
  elif "!=" in str_form:
    return 3
  elif "=" in str_form:
    return 4
  elif "in" in str_form:
    return 5
  else:
    return 6


all_sentences['sentence'] = all_sentences.apply(lambda row: clean_sent(row), axis = 1)
all_sentences['ops_label'] = all_sentences.apply(lambda row: label_formula_ops(row), axis = 1)
all_sentences['key_label'] = all_sentences.apply(lambda row: 1 if row['KeySentence'] == 1 else 0, axis = 1)



pg_data = all_sentences[all_sentences['dbms']=='pg']
ms_data = all_sentences[all_sentences['dbms']=='ms']


key_sentences = all_sentences[all_sentences['KeySentence']==1]
key_pg = key_sentences[key_sentences['dbms']=='pg']
key_ms = key_sentences[key_sentences['dbms']=='ms']


dbms_to_all = {'pg' : pg_data[['sentence', 'key_label']], 
               'ms' : ms_data[['sentence', 'key_label']]}
train_detection = pd.concat([dbms_to_all[d] for d in ['pg']])
test_detection = pd.concat([dbms_to_all[d] for d in ['ms']])
pd.set_option("display.max_rows", None, "display.max_columns", None)
print("Displaying Train and Test datasets details for detection:")
print(train_detection.info())
print(test_detection.info())

# Collect test and training data for classifying hints
dbms_to_class = {'pg' : key_pg[['sentence', 'ops_label']], 
                'ms' : key_ms[['sentence', 'ops_label']]}
train_classification = pd.concat([dbms_to_class[d] for d in ['pg']])
test_classification = pd.concat([dbms_to_class[d] for d in ['ms']])
print("Displaying Train and Test datasets for classification:")
print(train_classification.head())
print(test_classification.head())


def detect_base(sentence):
  if get_params(sentence) and get_vals(sentence):
    return True
  else:
    return False

def classify_base(sentence):
  vals = len(get_vals(sentence))
  if vals==1:
    return random.choice([0, 1, 2, 3, 4])
  elif vals>1:
    return 5
  else:
    return 6

print(f'Training: postgres ; Testing: mysql')
print('Performance(Matthews Coefficient and Confusion Matrix) of baseline for detecting key sentences :')
pred_detection = [detect_base(s) for s in test_detection['sentence']]
print(matthews_corrcoef(test_detection['key_label'], pred_detection))
print(confusion_matrix(test_detection['key_label'], pred_detection))


pred_classification = [classify_base(sent) for sent in test_classification['sentence']]
print('Performance(Matthews Coefficient and Confusion Matrix) of baseline for classifying key sentences :')
print(matthews_corrcoef(test_classification['ops_label'], pred_classification))
print(confusion_matrix(test_classification['ops_label'], pred_classification, labels=list(range(0,7))))


Displaying Train and Test datasets details for detection:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2980 entries, 0 to 2979
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence   2980 non-null   object
 1   key_label  2980 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 69.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3056 entries, 2980 to 6035
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence   3056 non-null   object
 1   key_label  3056 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 71.6+ KB
None
Displaying Train and Test datasets for classification:
                                              sentence  ops_label
196  Even on a modern Linux system, the stock kerne...          1
200  If you have a system with 1GB or more of RAM, ...          4
202  If you have less RAM you'll have to account m

In [23]:
print(train_detection)

                                               sentence  key_label
0           Performance Optimization - Postgre SQL wiki          0
1     Want to edit, but don't see an edit button whe...          0
2                                            Click here          0
3                                                                0
4                              Performance Optimization          0
5                                 From Postgre SQL wiki          0
6                           Jump to: navigation, search          0
7                                              Contents          0
8     1 How to Effectively Ask Questions Regarding P...          0
9                      2 General Setup and Optimization          0
10               3 Critical maintenance for performance          0
11                              4 Database architecture          0
12              5 Database Hardware Selection and Setup          0
13                                6 Benchmark Workloads       

In [24]:
print(test_detection)

                                               sentence  key_label
2980  10 essential My SQL performance tuning tips | ...          0
2981                                           Close Ad          0
2982                                          infoworld          0
2983                                      UNITED STATES          0
2984                                      United States          0
2985                                     United Kingdom          0
2986       Welcome! Here are the latest Insider stories          0
2987                                                             0
2988              Download Info World’s ultimate R data          0
2989                                  table cheat sheet          0
2990    14 technology winners and losers, post-COVID-19          0
2991  COVID-19 crisis accelerates rise of virtual ca...          0
2992  Q&A: Box CEO Aaron Levie looks at the future o...          0
2993                                       More Insider       

In [25]:
print(train_classification)

                                               sentence  ops_label
196   Even on a modern Linux system, the stock kerne...          1
200   If you have a system with 1GB or more of RAM, ...          4
202   If you have less RAM you'll have to account mo...          4
204   There are some workloads where even larger set...          1
206   Be aware that if your system or Postgre SQL bu...          1
211        On Windows the useful range is 64MB to 512MB          1
242   Setting effective_cache_size to 1/2 of total m...          0
263   Unless you're running on a very small configur...          2
265   For more write-heavy systems, values from 32 (...          0
269   Normally the large settings (>64/1GB) are only...          1
275   You can spread those writes out further, lower...          4
277   A setting of 0 gives something similar to the ...          2
289   However, it's acceptable to disable autovacuum...          4
314   Using all or mod in production would introduce...       

In [26]:
print(test_classification)

                                               sentence  ops_label
4552  The recommendation is to start with small valu...          1
4904  The following variables are largely dependent ...          0
4907  innodb_log_file_size This is generally set bet...          0
4912  Setting to “0” or “2” will give more performan...          5
4915  innodb_stats_on_metadata Setting this to “OFF”...          4
4916  innodb_buffer_pool_instances A best practice i...          4
4928  Other Settings Other Inno DB settings that can...          4
5017  For example, join_buffer_size set to 4GB when ...          3
5120  The following variables are largely dependent ...          0
5122  innodb_log_file_size This is generally set bet...          0
5125  Setting to “0” or “2” will give more performan...          5
5126  innodb_flush_method Setting this to O_DIRECT w...          4
5128  innodb_stats_on_metadata Setting this to “OFF”...          4
5129  innodb_buffer_pool_instances A best practice i...       

In [28]:
# Run this cell 2 times

model_args = ClassificationArgs(num_train_epochs=10, train_batch_size=50, overwrite_output_dir=True)
detection_model = ClassificationModel(model_type='roberta', use_cuda = True, model_name='roberta-base', args=model_args, num_labels=2, weight=[1, 85])
detection_model.args.no_save = True
detection_model.train_model(train_detection)


model_args = ClassificationArgs(num_train_epochs=20, train_batch_size=20, overwrite_output_dir=True)
type_model = ClassificationModel(model_type='roberta', use_cuda = True, model_name='roberta-base', args=model_args, num_labels=7)
type_model.args.no_save = True
type_model.train_model(train_classification)


key_prediction = detection_model.predict(list(test_detection['sentence']))
print('Evaluation for model detecting key sentences: ')
print(matthews_corrcoef(test_detection['key_label'], key_prediction[0]))
print(confusion_matrix(test_detection['key_label'], key_prediction[0]))


type_prediction = type_model.predict(list(test_classification['sentence']))
print('Evaluation for model classifying tuning hints:')
print(matthews_corrcoef(test_classification['ops_label'], type_prediction[0]))
print(confusion_matrix(test_classification['ops_label'], type_prediction[0], labels=list(range(0,7))))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/2980 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/60 [00:00<?, ?it/s]

...

  0%|          | 0/382 [00:00<?, ?it/s]

Evaluation for model detecting key sentences: 
0.5243538647253732
[[3006   15]
 [  17   18]]


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Evaluation for model classifying tuning hints:
0.3672106112349597
[[ 1  0  0  0  5  0  0]
 [ 0  2  1  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  1  0  0 21  0  0]
 [ 0  1  0  0  2  0  0]
 [ 0  0  0  0  0  0  0]]


In [29]:
def inference(test, dmodel, tmodel):
  db_hints = []
  key_prediction = dmodel.predict(list(test['sentence']))
  type_prediction = tmodel.predict(list(test['sentence']))
  for i in range(0, len(key_prediction[0])):
    if key_prediction[0][i] == 1:
      pred_type = type_prediction[0][i]    
      sentence = list(test['sentence'])[i]
      values = get_vals(sentence)
      context_idx = get_context(test, i)
      context_sent = test['sentence'].iloc[context_idx]
      params = set(get_params(context_sent))
      db_hints.append([sentence, context_sent, pred_type, params, values])
  return db_hints

In [30]:
db_hints = inference(test_detection, detection_model, type_model)

df_hints = pd.DataFrame(db_hints)
df_hints.columns = ['sentence', 'context', 'pred_type', 'params', 'values']
path = '/content/drive/My Drive/SDE_Project/hints_roberta_mysql.csv'
df_hints.to_csv(path, sep='\t')

  0%|          | 0/3056 [00:00<?, ?it/s]

  0%|          | 0/382 [00:00<?, ?it/s]

  0%|          | 0/3056 [00:00<?, ?it/s]

  0%|          | 0/382 [00:00<?, ?it/s]

#### References :



*   https://github.com/itrummer/dbbert
*   https://tinyurl.com/9crrjezv
*   https://dl.acm.org/doi/abs/10.1145/3503780.3503788
*   http://vldb.org/pvldb/vol14/p1159-trummer.pdf

